# Library

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install japanize_matplotlib
!pip install optuna
!pip install neologdn
import neologdn
import japanize_matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
import datetime as dt
import optuna.integration.lightgbm as lgbm
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import re

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)

     |████████████████████████████████| 4.1 MB 13.9 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256=57b9b2dbd04bf8c54eaadfb288a392190725ca2ac57e65d7eeeb2b5faeb2b6aa
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib
     |████████████████████████████████| 308 kB 13.9 MB/s 
     |████████████████████████████████| 209 kB 68.0 MB/s 
     |████████████████████████████████| 80 kB 8.7 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 111 kB 55.2 MB/s 
     |████████████████████████████████| 144 kB 69.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=8be440bdc4c12f779f7223489dcfdb93ed3539cdf42efbad550932f63f290bc3
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8

In [ ]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

import MeCab

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


Setting

In [ ]:
input_path = '/content/drive/MyDrive/input/nishika_novelist/'
output_path = '/content/drive/MyDrive/output/nishika_novelist/'
interm_path = '/content/drive/MyDrive/intermediate/nishika_novelist/'
now = dt.datetime.today()
month = str(now.month)
day = str(now.day)
competition_name = 'novelist_prediction'
comment = 'refuct'

save_nm = f'submission_{comment}_{month}_{day}.csv'
print(save_nm)

submission_refuct_11_3.csv


In [ ]:
STUDY_EDA = False
DEBUG = False

keyword_num = 300
title_num = 300
story_num = 300

# Load data

In [ ]:
train_df = pd.read_csv(input_path + 'train.csv')
test_df = pd.read_csv(input_path + 'test.csv')
sample_submission_df = pd.read_csv(input_path + 'sample_submission.csv')
data_explanation_df0 = pd.read_excel(input_path + 'data_explanation.xlsx', sheet_name = 0)
data_explanation_df1 = pd.read_excel(input_path + 'data_explanation.xlsx', sheet_name = 1)

In [ ]:
print(train_df.shape)
train_df.head(3)

(40000, 20)


,ncode,general_firstup,title,story,keyword,userid,writer,biggenre,genre,novel_type,end,isstop,isr15,isbl,isgl,iszankoku,istensei,istenni,pc_or_k,fav_novel_cnt_bin
0,N7588B,2007-04-01 16:40:57,暗い窓辺と皇帝の北の宮,※新しく手直ししました。\n\nhttp://ncode.syosetu.com/n2539l/\n\n宇宙開拓地の入植農家の子供として、小人の「僕」が生まれる。両親は我が子として受け入れよう...,ファンタジー SF 天使 小人 猫,9904,梅田浩志,4,402,1,0,0,0,0,0,0,0,0,0,1
1,N8726B,2007-04-22 15:40:30,仮想現実ゲーム 『ヴァルハラ』,ゲームセンターの片隅に置かれた律儀なまでにリアルなゲーム、ヴァルハラ。それはプレイを重ねるたびに俺たちを魅了していった。そして今日も勇士《エイン》たちの戦いは終わらない。――戦場で育まれる戦友...,仮想現実 ゲーム バトル 戦乙女 高校生 学校/学園 アンドロイド ライト SF エンターテイメント ハイテク 友情 現代(モダン) 未来,6527,ルト,4,401,1,0,0,0,0,0,0,0,0,3,2
2,N5481C,2007-08-22 18:25:54,海の竜騎兵,海軍の中に存在するクジラ部隊「竜騎兵」 主人公はその一員だ。\n騎乗する鯨との間に結ばれた絆は強く、主人公の戦闘能力も次第に洗練されてゆく。\n敵国にライバルが出現。新兵器の登場。\nすばしっ...,ヒーロー 冒険 海軍 深海 海中の戦闘 HJ2021,13882,雨宮雨彦,4,403,1,0,0,0,0,0,0,0,0,0,3


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ncode              40000 non-null  object
 1   general_firstup    40000 non-null  object
 2   title              40000 non-null  object
 3   story              40000 non-null  object
 4   keyword            37774 non-null  object
 5   userid             40000 non-null  int64 
 6   writer             40000 non-null  object
 7   biggenre           40000 non-null  int64 
 8   genre              40000 non-null  int64 
 9   novel_type         40000 non-null  int64 
 10  end                40000 non-null  int64 
 11  isstop             40000 non-null  int64 
 12  isr15              40000 non-null  int64 
 13  isbl               40000 non-null  int64 
 14  isgl               40000 non-null  int64 
 15  iszankoku          40000 non-null  int64 
 16  istensei           40000 non-null  int64

In [ ]:
train_df['train_test'] = 'train'
test_df['train_test'] = 'test'
train_test = pd.concat([train_df, test_df], axis = 0).reset_index(drop = True)

In [ ]:
train_test.head(3)

,ncode,general_firstup,title,story,keyword,userid,writer,biggenre,genre,novel_type,end,isstop,isr15,isbl,isgl,iszankoku,istensei,istenni,pc_or_k,fav_novel_cnt_bin,train_test
0,N7588B,2007-04-01 16:40:57,暗い窓辺と皇帝の北の宮,※新しく手直ししました。\n\nhttp://ncode.syosetu.com/n2539l/\n\n宇宙開拓地の入植農家の子供として、小人の「僕」が生まれる。両親は我が子として受け入れよう...,ファンタジー SF 天使 小人 猫,9904,梅田浩志,4,402,1,0,0,0,0,0,0,0,0,0,1.0,train
1,N8726B,2007-04-22 15:40:30,仮想現実ゲーム 『ヴァルハラ』,ゲームセンターの片隅に置かれた律儀なまでにリアルなゲーム、ヴァルハラ。それはプレイを重ねるたびに俺たちを魅了していった。そして今日も勇士《エイン》たちの戦いは終わらない。――戦場で育まれる戦友...,仮想現実 ゲーム バトル 戦乙女 高校生 学校/学園 アンドロイド ライト SF エンターテイメント ハイテク 友情 現代(モダン) 未来,6527,ルト,4,401,1,0,0,0,0,0,0,0,0,3,2.0,train
2,N5481C,2007-08-22 18:25:54,海の竜騎兵,海軍の中に存在するクジラ部隊「竜騎兵」 主人公はその一員だ。\n騎乗する鯨との間に結ばれた絆は強く、主人公の戦闘能力も次第に洗練されてゆく。\n敵国にライバルが出現。新兵器の登場。\nすばしっ...,ヒーロー 冒険 海軍 深海 海中の戦闘 HJ2021,13882,雨宮雨彦,4,403,1,0,0,0,0,0,0,0,0,0,3.0,train


# EDA

In [ ]:
if STUDY_EDA:
    display(data_explanation_df0.head())
    display(data_explanation_df1.head(30))

In [ ]:
if STUDY_EDA:
    for col_nm in train_test.columns:
        
        print(col_nm)
        print(train_test[col_nm].nunique())
        print(train_test[col_nm].unique())
        print('*'*100)
        print('*'*100)
        print('*'*100)

#Preprossing

## general_firstup

In [ ]:
train_test['general_firstup'] = pd.to_datetime(train_test['general_firstup'], format='%Y-%m-%d').dt.date
print(train_test['general_firstup'].max())
print(train_test['general_firstup'].min())

2021-09-23
2007-04-01


In [ ]:
train_test['date_fromup'] = train_test['general_firstup'].max() - train_test['general_firstup']
train_test['date_fromup'] = train_test['date_fromup'].dt.days

## userid, writer

In [ ]:
writer_df = train_test[['userid', 'writer']].drop_duplicates()
writernum_df = pd.DataFrame(writer_df['userid'].value_counts())
writernum_df = writernum_df.rename(columns = {'userid':'userid_double_count'})
writernum_df['userid'] = writernum_df.index
writernum_df = writernum_df.reset_index(drop = True)
writernum_df.head()

,userid_double_count,userid
0,88,1275419
1,24,815315
2,21,1098859
3,9,663566
4,8,337972


In [ ]:
print(train_test.shape)

train_test = pd.merge(
    train_test,
    writernum_df,
    on = 'userid',
    how = 'left'
)

print(train_test.shape)

(48522, 22)
(48522, 23)


In [ ]:
train_test['userid_count'] = train_test.groupby('userid')['userid'].transform('count')
train_test['writer_count'] = train_test.groupby('writer')['writer'].transform('count')

## normalize word

## keyword, title, **story**

In [ ]:
train_test['story'] = train_test['story'].apply(lambda x:neologdn.normalize(x))

train_test['story'] = train_test['story'].apply(lambda x :re.sub(
    r'(http|https)://([-\w]+\.)+[-\w]+(/[-\w./?%&=]*)?', 
    "", 
    x
))

train_test['story'] = train_test['story'].apply(lambda x:x.replace('\n', ''))


train_test['title'] = train_test['title'].apply(lambda x:neologdn.normalize(x))
train_test['title'] = train_test['title'].apply(lambda x :re.sub(
    r'(http|https)://([-\w]+\.)+[-\w]+(/[-\w./?%&=]*)?', 
    "", 
    x
))

train_test['title'] = train_test['title'].apply(lambda x:x.replace('\n', ''))

In [ ]:
train_test['length_title'] = train_test['title'].apply(lambda x:len(x))
train_test['length_story'] = train_test['story'].apply(lambda x:len(x))
train_test['length_keyword'] = train_test['keyword'].astype(str).apply(lambda x:len(x))

### keyword

In [ ]:
# 人気度的なデータを生成
train_test['keyword'] = train_test['keyword'].astype(str)
keyword_list = set()
keyword_dict = dict()

for i in range(len(train_test)): 
    print("\r"+str(i),end="")
    for keyword in train_test['keyword'].iloc[i].replace('\u3000', ' ').split(' '):
        
        if keyword != '':
            keyword_list.add(str(keyword))
            
            try:
                keyword_dict[keyword] += 1
            except:
                keyword_dict[keyword] = 0


48521

In [ ]:
for k, v in list(keyword_dict.items()):
    if v <= keyword_num:
      keyword_dict.pop(k, None)
      keyword_list.remove(k)

In [ ]:
print(train_test.shape)

# カラムを作成。初期値0
for keyword in keyword_list:
    train_test[keyword] = 0

print(train_test.shape)

(48522, 28)
(48522, 150)


In [ ]:
# 人気度的なデータを生成
for i in range(len(train_test)):
    print("\r"+str(i),end="")
    for keyword in train_test['keyword'].iloc[i].replace('\u3000', ' ').split(' '):
        
        if (keyword != '') & (keyword in train_test.columns):
            train_test[keyword].iloc[i] = keyword_dict[keyword]

16

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



48521

In [ ]:
# 人気度合計カラムを生成
train_test['keyword_sum'] = 0
for keyword in keyword_list:
    train_test['keyword_sum'] += train_test[keyword]

### title

In [ ]:
tagger = MeCab.Tagger('-Ochasen')

print(train_test.shape)

noun_title_dict = dict()
noun_title_list = set()

for i in range(len(train_test)): 
    print("\r"+str(i),end="")
    title_text = train_test['title'].iloc[i]

    nouns = [line.split()[0] for line in tagger.parse(title_text).splitlines()
               if "名詞" in line.split()[-1]]

    for noun in nouns:
        noun_title_list.add(noun + '_title')

        if noun + '_title' in noun_title_dict.keys():
            noun_title_dict[noun + '_title'] += 1
        else:
            noun_title_dict[noun + '_title'] = 0

for k, v in list(noun_title_dict.items()):
    if v <= title_num:
      noun_title_dict.pop(k, None)
      noun_title_list.remove(k)

# カラムを作成。初期値0
for noun_title in noun_title_list:
    train_test[noun_title] = 0

print(train_test.shape)

(48522, 151)
48521(48522, 228)


In [ ]:
# 人気度的なデータを生成
for i in range(len(train_test)):
    print("\r"+str(i),end="")
    title_text = train_test['title'].iloc[i]

    nouns = [line.split()[0] for line in tagger.parse(title_text).splitlines()
               if "名詞" in line.split()[-1]]

    for noun in nouns:
        if noun + '_title' in train_test.columns:
            train_test[noun + '_title'].iloc[i] = noun_title_dict[noun + '_title']

# 人気度合計カラムを生成
train_test['title_sum'] = 0
for noun_title in noun_title_list:
    train_test['title_sum'] += train_test[noun_title]

1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



48521

### story

In [ ]:
tagger = MeCab.Tagger('-Ochasen')

print(train_test.shape)

noun_story_dict = dict()
noun_story_list = set()

for i in range(len(train_test)): 
    print("\r"+str(i),end="")
    story_text = train_test['story'].iloc[i]

    nouns = [line.split()[0] for line in tagger.parse(story_text).splitlines()
               if "名詞" in line.split()[-1]]

    for noun in nouns:
        noun_story_list.add(noun + '_story')

        if noun + '_story' in noun_story_dict.keys():
            noun_story_dict[noun + '_story'] += 1
        else:
            noun_story_dict[noun + '_story'] = 0

for k, v in list(noun_story_dict.items()):
    if v <= story_num:
      noun_story_dict.pop(k, None)
      noun_story_list.remove(k)

# カラムを作成。初期値0
for noun_story in noun_story_list:
    train_test[noun_story] = 0

print(train_test.shape)

(48522, 229)
48521(48522, 2115)


In [ ]:
# 人気度的なデータを生成
for i in range(len(train_test)):
    print("\r"+str(i),end="")
    story_text = train_test['story'].iloc[i]

    nouns = [line.split()[0] for line in tagger.parse(story_text).splitlines()
               if "名詞" in line.split()[-1]]

    for noun in nouns:
        if noun + '_story' in train_test.columns:
            train_test[noun + '_story'].iloc[i] = noun_story_dict[noun + '_story']

# 人気度合計カラムを生成
train_test['story_sum'] = 0
for noun_story in noun_story_list:
    train_test['story_sum'] += train_test[noun_story]

0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



48521

## label_encode

In [ ]:
label_cols = ['userid', 'writer']

le = LabelEncoder()

for label_col in label_cols:
    le = LabelEncoder()
    train_test[label_col] = le.fit_transform(train_test[label_col])

In [ ]:
train_test.head(3)

,ncode,general_firstup,title,story,keyword,userid,writer,biggenre,genre,novel_type,end,isstop,isr15,isbl,isgl,iszankoku,istensei,istenni,pc_or_k,fav_novel_cnt_bin,train_test,date_fromup,userid_double_count,userid_count,writer_count,length_title,length_story,length_keyword,西洋,ゲラゲラコンテスト３,日常,古典恋愛,ネット小説大賞九,探偵小説,123大賞,シリアス,エイリアン,バイオハザード,悲恋,ホームドラマ,童話,天災,ゲーム,ミステリー,主人公最強,人工知能,サスペンス,異世界転生,ESN大賞３,乙女ゲーム,...,執筆_story,思い_story,表現_story,過去_story,領主_story,リンク_story,髪_story,簡単_story,対象_story,:_story,おかげ_story,太_story,計画_story,とき_story,集_story,確か_story,ぶり_story,PV_story,間_story,黒_story,団体_story,12_story,結_story,家_story,小学校_story,普段_story,ひとつ_story,一族_story,非常_story,学校_story,員_story,喪失_story,マリア_story,観_story,多く_story,翼_story,妻_story,線_story,事態_story,設定_story,斗_story,故郷_story,アメリカ_story,会議_story,侍女_story,前作_story,世界中_story,特定_story,春_story,story_sum
0,N7588B,2007-04-01,暗い窓辺と皇帝の北の宮,※新しく手直ししました。宇宙開拓地の入植農家の子供として、小人の「僕」が生まれる。両親は我が子として受け入れようとし、飼い猫のロマノフも「僕」の世話をしてくれる。しかし、子供達は受け入れてはく...,ファンタジー SF 天使 小人 猫,44,13724,4,402,1,0,0,0,0,0,0,0,0,0,1.0,train,5289,1,3,3,11,211,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32157
1,N8726B,2007-04-22,仮想現実ゲーム『ヴァルハラ』,ゲームセンターの片隅に置かれた律儀なまでにリアルなゲーム、ヴァルハラ。それはプレイを重ねるたびに俺たちを魅了していった。そして今日も勇士《エイン》たちの戦いは終わらない。ー戦場で育まれる戦友達...,仮想現実 ゲーム バトル 戦乙女 高校生 学校/学園 アンドロイド ライト SF エンターテイメント ハイテク 友情 現代(モダン) 未来,27,7714,4,401,1,0,0,0,0,0,0,0,0,3,2.0,train,5268,1,2,3,14,112,69,0,0,0,0,0,0,0,0,0,0,0,0,0,0,775,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32134
2,N5481C,2007-08-22,海の竜騎兵,海軍の中に存在するクジラ部隊「竜騎兵」主人公はその一員だ。騎乗する鯨との間に結ばれた絆は強く、主人公の戦闘能力も次第に洗練されてゆく。敵国にライバルが出現。新兵器の登場。すばしっこく泳ぐ鯨に魚...,ヒーロー 冒険 海軍 深海 海中の戦闘 HJ2021,68,19098,4,403,1,0,0,0,0,0,0,0,0,0,3.0,train,5146,1,9,9,5,121,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,957,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17635


# Model

In [ ]:
train_test.columns.tolist()

['ncode',
 'general_firstup',
 'title',
 'story',
 'keyword',
 'userid',
 'writer',
 'biggenre',
 'genre',
 'novel_type',
 'end',
 'isstop',
 'isr15',
 'isbl',
 'isgl',
 'iszankoku',
 'istensei',
 'istenni',
 'pc_or_k',
 'fav_novel_cnt_bin',
 'train_test',
 'date_fromup',
 'userid_double_count',
 'userid_count',
 'writer_count',
 'length_title',
 'length_story',
 'length_keyword',
 '西洋',
 'ゲラゲラコンテスト３',
 '日常',
 '古典恋愛',
 'ネット小説大賞九',
 '探偵小説',
 '123大賞',
 'シリアス',
 'エイリアン',
 'バイオハザード',
 '悲恋',
 'ホームドラマ',
 '童話',
 '天災',
 'ゲーム',
 'ミステリー',
 '主人公最強',
 '人工知能',
 'サスペンス',
 '異世界転生',
 'ESN大賞３',
 '乙女ゲーム',
 '史実',
 'IF戦記',
 'アンドロイド',
 '学園',
 'ボーイズラブ',
 'ざまあ',
 '男主人公',
 '宇宙',
 'アニメイト耳聴き1',
 'タイムマシン',
 '溺愛',
 '魔王',
 '現代',
 '二次創作',
 '冬童話2021',
 'ヒーロー',
 '猫',
 '友情',
 'ほのぼの',
 '魔法',
 '私小説',
 'ディストピア',
 '悪役令嬢',
 'アクション',
 '婚約破棄',
 '異世界',
 'ホラー',
 'オリジナル戦記',
 '伝奇',
 'ハーレム',
 'ロボット',
 '新人発掘コンテスト',
 '電脳世界',
 'ファンタジー',
 '追放',
 '身分差',
 'ガールズラブ',
 '怪獣',
 'ギャグ',
 'パンデミック',
 '冒険',
 '百合',
 'ネット小説大賞九感想',
 'ダンジョン',
 '女主人公

In [ ]:
drop_cols = [
    'ncode',
    'general_firstup',
    'title',
    'story',
    'keyword'
 ]

In [ ]:
train_test = train_test.drop(columns = drop_cols)
train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48522 entries, 0 to 48521
Columns: 2111 entries, userid to story_sum
dtypes: float64(1), int64(2109), object(1)
memory usage: 781.8+ MB


In [ ]:
target = 'fav_novel_cnt_bin'

preprocessed_train_df = train_test[train_test['train_test'] == 'train'].drop(columns = 'train_test')
preprocessed_test_df = train_test[train_test['train_test'] == 'test'].drop(columns = ['train_test', target])

if DEBUG == True:
    x = preprocessed_train_df.drop(columns = target)[:500]
    y = preprocessed_train_df[target][:500]

else:
    x = preprocessed_train_df.drop(columns = target)
    y = preprocessed_train_df[target]

In [ ]:
# foldデータ作成
fold = KFold(n_splits=5, shuffle=True, random_state=42)
# fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv = list(fold.split(x, y))

In [ ]:
def fit_lgbm(x, y, cv, params: dict=None, verbose=100):
        
    oofs_lgbm = []
    models_lgbm = []
    scores_lgbm = []
    
    for i, (idx_train, idx_valid) in enumerate(cv): 
        
        # この部分が交差検証のところです。データセットを cv instance によって分割します
        # training data を trian/valid に分割
        x_train, y_train = x[idx_train], y[idx_train]
        x_valid, y_valid = x[idx_valid], y[idx_valid]
        
        oof_pred = np.zeros(len(y_valid), dtype=np.float32)
        
        trains = lgbm.Dataset(x_train, y_train)
        valids = lgbm.Dataset(x_valid, y_valid)

        model = lgbm.train(
            params, 
            trains,
            valid_sets = valids,
            num_boost_round = 50000,
            verbose_eval = False,
            early_stopping_rounds = 100
        )

        
        oof_pred = model.predict(x_valid)
        # score = log_loss(oof_pred, y_valid)

        print('*'*50)
        print('*'*50)
        print('*'*50)
        
        print(f'fold{i+1}finished')
        # print('rmse:', score)
        
        print('*'*50)
        print('*'*50)
        print('*'*50)
        
        oofs_lgbm.append(oof_pred)
        models_lgbm.append(model)
        # scores_lgbm.append(score)
        
    return oofs_lgbm, models_lgbm
    # , scores_lgbm

params = {
    'objective':'multiclass',
    'num_class': 5,
    'metric':'multi_logloss'
}

oofs_lgbm, models_lgbm = fit_lgbm(
    x.values, 
    y,
    cv,
    params
)

[I 2021-11-03 00:29:05,362] A new study created in memory with name: no-name-355d494a-374b-4663-ba6a-bc50a6d87b9a
feature_fraction, val_score: 0.808878:  57%|#####7    | 4/7 [02:03<01:34, 31.67s/it]

KeyboardInterrupt: ignored

In [ ]:
def visualize_importance(models, feat_train_df):
    """lightGBM の model 配列の feature importance を plot する
    CVごとのブレを boxen plot として表現します.
    
    args:
        models:
            List of lightGBM models
        feat_train_df:
            学習時に使った DataFrame
    """
    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importance()
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(max(6, len(order) * .4), 7))
    sns.boxenplot(data=feature_importance_df, x='column', y='feature_importance', order=order, ax=ax, palette='viridis')
    ax.tick_params(axis='x', rotation=90)
    ax.grid()
    fig.tight_layout()
    return fig, ax

In [ ]:
fig, ax = visualize_importance(models_lgbm, x)

In [ ]:
y_pred_lgbm = np.array([model.predict(preprocessed_test_df.values) for model in models_lgbm])
y_pred_lgbm = np.mean(y_pred_lgbm, axis=0)
# y_pred_lgbm = np.where(y_pred_lgbm < 0, 0, y_pred_lgbm)

In [ ]:
y_pred_lgbm

In [ ]:
sample_submission_df.head()

In [ ]:
pred_df = pd.DataFrame(y_pred_lgbm, columns = ['proba_0', 'proba_1', 'proba_2', 'proba_3', 'proba_4'])
pred_df

In [ ]:
sample_submission_df['proba_0'] = pred_df['proba_0']
sample_submission_df['proba_1'] = pred_df['proba_1']
sample_submission_df['proba_2'] = pred_df['proba_2']
sample_submission_df['proba_3'] = pred_df['proba_3']
sample_submission_df['proba_4'] = pred_df['proba_4']

sample_submission_df.head()

In [ ]:
sample_submission_df.tail()

In [ ]:
sample_submission_df.to_csv(output_path + save_nm, index = False)

In [ ]:
y_confirm_lgbm = np.array([model.predict(x.values) for model in models_lgbm])
y_confirm_lgbm = np.mean(y_confirm_lgbm, axis=0)

In [ ]:
confirm_df = pd.DataFrame(y_confirm_lgbm, columns = ['proba_0', 'proba_1', 'proba_2', 'proba_3', 'proba_4'])
confirm_df = pd.concat([preprocessed_train_df, confirm_df], axis = 1)
confirm_df.head(3)

In [ ]:
ext_cols = list(set(confirm_df.columns) & set(train_df.columns))

In [ ]:
ext_cols += ['proba_0', 'proba_1', 'proba_2', 'proba_3', 'proba_4', 'date_fromup', 'length_title', 'length_story', 'length_keyword', 'userid_count', 'keyword_sum']

In [ ]:
ext_cols

In [ ]:
confirm_df2 = confirm_df.copy()[ext_cols]

In [ ]:
eval0 = confirm_df2[confirm_df2['fav_novel_cnt_bin'] == 0]

In [ ]:
for ii in [0, 1, 2, 3, 4]:
    print(ii)
    
    tmp_df = confirm_df2[confirm_df2['fav_novel_cnt_bin'] == ii]


    if ii == 0:
        sns.boxplot(x = 'fav_novel_cnt_bin', y = 'proba_0', data = tmp_df)
        plt.ylim(0, 1)
        plt.show()

        
    elif ii == 1:
        sns.boxplot(x = 'fav_novel_cnt_bin', y = 'proba_1', data = tmp_df)
        plt.ylim(0, 1)
        plt.show()

        
    elif ii == 2:
        sns.boxplot(x = 'fav_novel_cnt_bin', y = 'proba_2', data = tmp_df)
        plt.ylim(0, 1)
        plt.show()

        
    elif ii == 3:
        sns.boxplot(x = 'fav_novel_cnt_bin', y = 'proba_3', data = tmp_df)
        plt.ylim(0, 1)
        plt.show()


    elif ii == 4:
        sns.boxplot(x = 'fav_novel_cnt_bin', y = 'proba_4', data = tmp_df)
        plt.ylim(0, 1)
        plt.show()


    print('*'*100)
    print('*'*100)
    print('*'*100)
    print('*'*100)

In [ ]:
# https://datumstudio.jp/blog/python%E3%81%AB%E3%82%88%E3%82%8B%E6%97%A5%E6%9C%AC%E8%AA%9E%E5%89%8D%E5%87%A6%E7%90%86%E5%82%99%E5%BF%98%E9%8C%B2/